In [2]:
# a little script to reconcile map and camera images
import os
import pandas as pd
import cv2

path1 = 'C:/SelfDrive/Map Projection/map_img/'
path2 = 'C:/SelfDrive/Map Projection/img/'

images = [f.split('.png')[0] for f in os.listdir(path2) if f.endswith(".png")]
maps = [f.split('.png')[0] for f in os.listdir(path1) if f.endswith(".png")]

im_df = pd.DataFrame(images)
map_df = pd.DataFrame(maps)

in_both = pd.merge(left=im_df,right=map_df,on=0)

print('maps: ',len(maps), '  images: ',len(images),'  in both: ',len(in_both))

maps:  64   images:  4   in both:  1


In [13]:
# This section finds row index when green tip of the car is located on the map
# so it could be used to crop all images to achive consistent location of the map

maps = [f for f in os.listdir(path1) if f.endswith(".png")]
sample_map = cv2.imread(path1+maps[0],cv2.IMREAD_COLOR)
# get a section of image with the car's outline
slice_map = sample_map
#finding green colour in image
hsv_img = cv2.cvtColor(slice_map, cv2.COLOR_BGR2HSV)
lower_range = (40, 40, 40) # lower range of green
upper_range = (70, 255, 255) # upper range of green
mask = cv2.inRange(hsv_img, lower_range, upper_range)
#color_image = cv2.bitwise_and(slice_map, slice_map, mask=mask)
#cv2.imshow('Color Image', color_image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

region_to_look_y = slice_map.shape[0]-80
region_to_look_x1 = int(slice_map.shape[1]/2-10)
region_to_look_x2 = int(slice_map.shape[1]/2+10)

crop = mask[region_to_look_y:, region_to_look_x1:region_to_look_x2]
point_loc = 0
for i in range(crop.shape[0]):
    if crop[i,].sum() > 0 and point_loc==0:
        print("Detected point of arow in row ",i)
        point_loc = i
row_of_arrow = region_to_look_y + point_loc
print("Index of row in original map of arrow tip is: ",row_of_arrow)